# 판다스 3편

## 주요 내용

`Series`와 `DataFrame` 객체를로부터 기초 통계 자료를 추출하는 방식을 다룬다.

* 합, 누적합
* 상관관계, 공분산
* 중복값 처리

## 기본 설정

`pandas` 라이브러리는 보통 `pd` 라는 별칭으로 사용된다.

In [1]:
import pandas as pd
import numpy as np

랜덤 시드, 어레이 내부에 사용되는 부동소수점 정확도, 도표 크기 지정 옵션 등은 이전과 동일하다.

In [2]:
np.random.seed(12345)
np.set_printoptions(precision=4, suppress=True)

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

`Series`와 `DataFrame`을 표로 보여줄 때 사용되는 행의 수를 20으로 지정한다. 
기본 값은 60이다.

In [3]:
PREVIOUS_MAX_ROWS = pd.options.display.max_rows # 원래 60이 기본.
pd.set_option("max_rows", 20)

## 5.3 기초 통계 함수 (p. 226)

기초 통계에서 사용되는 주요 메서드들의 활용법을 살펴본다.

* `sum()`
* `mean()`
* `std()`
* `idxmax()`/`idxmin()`
* `cumsum()`
* `describe()`

기본적으로 열 단위로 작동하며, 결측치는 행 또는 열의 모든 값이 결측치가 아니라면 기본적으로 무시된다.
행 단위로 작동하게 하려면 축을 `axis=1` 또는 `axis='columns`로 지정하고,
결측치를 무시하지 않으려면 `skipna=False`로 지정한다.

In [4]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5],
                   [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


* `sum()` 메서드: 행/열 단위 합 계산

In [5]:
df.sum()

one    9.25
two   -5.80
dtype: float64

결측치를 무시하지 않으면, 결측치가 포함된 행/렬에 대한 계산은 하지 않는다.

In [6]:
df.sum(skipna=False)

one   NaN
two   NaN
dtype: float64

In [7]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

시리즈는 하나의 열을 갖는 데이터프레임처럼 작동한다.

In [8]:
df['one']

a    1.40
b    7.10
c     NaN
d    0.75
Name: one, dtype: float64

In [9]:
df['one'].sum()

9.25

* `mean()` 메서드: 평균값 계산

In [10]:
df.mean()

one    3.083333
two   -2.900000
dtype: float64

In [11]:
df.mean(axis='columns')

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

결측치를 무시하지 않으면, 결측치가 포함된 행/렬에 대한 계산은 하지 않는다.

In [12]:
df.mean(skipna=False)

one   NaN
two   NaN
dtype: float64

In [13]:
df.mean(axis='columns', skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

시리즈의 경우도 동일하게 작동한다.

In [14]:
df['one'].mean()

3.0833333333333335

In [15]:
df['one'].mean(skipna=False)

nan

* `std()` 메서드: 평균값 계산

In [16]:
df.std()

one    3.493685
two    2.262742
dtype: float64

In [17]:
df.std(axis='columns')

a         NaN
b    8.202439
c         NaN
d    1.449569
dtype: float64

In [18]:
df.std(skipna=False)

one   NaN
two   NaN
dtype: float64

In [19]:
df.std(axis='columns', skipna=False)

a         NaN
b    8.202439
c         NaN
d    1.449569
dtype: float64

* `idxmax()`/`idxmin()`: 최댓값/최솟값을 갖는 인덱스 확인

아래 코드는 열별 최댓값을 갖는 인덱스를 찾아준다.

In [20]:
df.idxmax()

one    b
two    d
dtype: object

아래 코드는 행별 최솟값을 갖는 인덱스를 찾아준다.

In [21]:
df.idxmin(axis=1)

a    one
b    two
c    NaN
d    two
dtype: object

* `cumsum()`: 누적 합 계산

In [22]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [23]:
df.cumsum(skipna=False)

,one,two
a,1.4,NaN
b,8.5,NaN
c,NaN,NaN
d,NaN,NaN


* `describe()`: 요약 통계 보여주기

수치형 데이터의 경우 평균값, 표준편차, 사분위수 등의 통계 정보를 요약해서 보여준다.

In [24]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


수치형 데이터가 아닐 경우 다른 요약 통계를 보여준다.

In [25]:
ser = pd.Series(['a', 'a', 'b', 'c'] * 2)
ser

0    a
1    a
2    b
3    c
4    a
5    a
6    b
7    c
dtype: object

In [26]:
ser.describe()

count     8
unique    3
top       a
freq      4
dtype: object

### 5.3.1 상관관계와 공분산 (p. 229)

금융 사이트에서 구한 4 개 회사의 주가(price)와 거래량(volume)을 담고 있는 두 개의 데이터를 이용하여
상관계수와 공분산을 계산해본다.

먼저 두 개의 데이터를 불러온다.

- pkl 파일: 판다스에서 제공하는 객체를 `to_pickle()` 메서드를 이용하여 
    컴퓨터에 파일로 저장할 때 사용되는 바이너리 파일.
- `read_pickle()`: 저장된 pkl 파일을 파이썬으로 불러오는 함수

아래 코드는 일별 주가 데이터를 불러온다.
2010년 1월 4일부터 2016년 10월 21일까지의 데이터 1714개를 담고 있다.

In [27]:
price = pd.read_pickle('examples/yahoo_price.pkl')
price

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...
2016-10-17,117.550003,779.960022,154.770004,57.220001
2016-10-18,117.470001,795.260010,150.720001,57.660000
2016-10-19,117.120003,801.500000,151.259995,57.529999


아래 코드는 동일 회사, 동일 날짜의 1일 거래량(volume) 담고 있는 데이터를 불러온다.

In [28]:
volume = pd.read_pickle('examples/yahoo_volume.pkl')
volume

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400
...,...,...,...,...
2016-10-17,23624900,1089500,5890400,23830000
2016-10-18,24553500,1995600,12770600,19149500
2016-10-19,20034600,116600,4632900,22878400


주가의 일단위 변화율을 알아보기 위해 퍼센트 변화율을 확인해보자.

__참고:__ 증권분야에서 return은 이익율을 의미한다.

In [29]:
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


####  `corr()`/`cov()` 메서드

상관계수와 공분산 모두 두 확률변수 사이의 선형관계를 보여주며
차이점은 다음과 같다.

- 공분산: 두 확률변수 $X, Y$ 사이의 선형관계를 계량화 함. 
    양수/음수 여부에 따라 양 또는 음의 선형관계이며,
    절댓값이 클 수록 강한 선형관계임.
    다만, 사용되는 확률변수의 척도(scale)에 많은 영향을 받음.
    따라서 보통 정규화한 값인 상관계수를 사용함.

$$
\begin{align*}
Cov(X, Y) & = E((X-\mu_X)(Y-\mu_Y))\\[2ex]
\mu_X & = E(X) = \dfrac{\sum X}{n}\\[1.5ex]
\mu_Y & = E(Y) = \dfrac{\sum Y}{n}
\end{align*}
$$

- 상관계수: 두 확률변수 사이의 선형관계를 -1과 1 사이의 값으로 표현.
    양수/음수 여부에 따라 양 또는 음의 선형관계이며,
    절댓값이 1에 가까울 수록 강한 선형관계임.
$$
\begin{align*}
\rho & = \frac{Cov(X, Y)}{\sigma_X\cdot \sigma_Y}\\[2ex]
\sigma_X & = \sqrt{Var(X)}\\[1.5ex]
\sigma_X & = \sqrt{Var(X)}\\[1.5ex]
Var(X) & = \dfrac{\sum (X-\mu_X)^2}{n}\\[1.5ex]
Var(Y) & = \dfrac{\sum (X-\mu_Y)^2}{n}
\end{align*}
$$    

'MSFT'와 'IBM' 사이의 상관계수는 다음과 같다.

In [30]:
returns['MSFT'].corr(returns['IBM'])

0.4997636114415114

'MSFT'와 'IBM' 사이의 공분산은 다음과 같다.

In [31]:
returns['MSFT'].cov(returns['IBM'])

8.870655479703546e-05

아래와 같이 속성으로 처리할 수 있으면 보다 깔금하게 보인다.

In [32]:
returns.MSFT.corr(returns.IBM)

0.4997636114415114

전체 회사를 대상으로 하는 상관계수와 공분산을 계산할 수도 있다.

In [33]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


In [34]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


#### `corrwith()` 메서드: 다른 시리즈 또는 데이터프레임과의 상관계수 계산

시리즈를 인자로 사용하면 각 열에 대한 상관계수를 계산한다.

In [35]:
returns.corrwith(returns.IBM)

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

데이터프레임을 인자로 사용하면 공통 인덱스를 사용하는 모든 열에 대한 상관계수를 계산한다.

In [36]:
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

### 5.3.2 중복과 빈도수 (p. 232)

#### `unique()` 메서드

시리즈에서 사용된 값을 중복 없이 확인하려면 `unique()` 메서드를 이용한다.
`set()` 함수와 유사하게 작동하며, 넘파이 어레이를 반환한다.

In [37]:
obj = pd.Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [38]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

#### `value_counts()` 메서드

값들의 빈도수를 확인하기 위해 사용한다.

In [39]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

In [40]:
pd.value_counts(obj.values, sort=False)

a    3
c    3
d    1
b    2
dtype: int64

## 연습문제

아래 코드는 인터넷 데이터 저장소로부터 아이리스(붓꽃) 데이터(`iris.data`)를 
2차원 넘파이 어레이로 불러온다.

In [41]:
# 아이리스(붓꽃) 데이터 불러오기
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
iris = np.genfromtxt(url, delimiter=',', dtype='str')

`iris.data` 파일에는 아래 형식의 데이터가 150개 들어 있다. 

```python
5.1,3.5,1.4,0.2,Iris-setosa
```

하나의 데이터에 사용된 값들은 하나의 아이리스(붓꽃)에 대한 꽃잎, 꽃받침과 관련된 특성(features)과 품종을 나타내며,
보다 구체적으로 아래 순서를 따른다.

```
꽃받침 길이, 꽃받침 너비, 꽃잎 길이, 꽃잎 너비, 품종
```

In [42]:
type(iris)

numpy.ndarray

In [43]:
iris.shape

(150, 5)

길이와 너비를 저장하는 특성들은 숫자로 저장되어 있었지만 위 코드는 문자열로 저장된 품종 특성과의 자료형을 통일시키기 위해
모두 문자열 자료형으로 불러왔다.
처음 5개 데이터를 확인하면 다음과 같다.

__참고:__ `'<U15'`는 길이가 최대 15인 유니코드 문자열 자료형을 나타낸다.

In [44]:
iris[:5]

array([['5.1', '3.5', '1.4', '0.2', 'Iris-setosa'],
       ['4.9', '3.0', '1.4', '0.2', 'Iris-setosa'],
       ['4.7', '3.2', '1.3', '0.2', 'Iris-setosa'],
       ['4.6', '3.1', '1.5', '0.2', 'Iris-setosa'],
       ['5.0', '3.6', '1.4', '0.2', 'Iris-setosa']], dtype='<U15')

수치형 데이터와 품종 데이터를 분리해서 각각 (150,4), (150,) 모양의 어레이를 생성하자.
이때 수치형 데이터는 `'f8'`, 즉 `'float64'` 자료형을 사용하도록 한다.

In [45]:
iris_features = iris[:,:4].astype('f8')
iris_labels = iris[:, 4]

두 어레이를 판다스의 데이터프레임으로 형변환한다.
이때 각 열의 이름을 사용된 데이터 특성을 반영하도록 지정한다.

In [46]:
columns = ['꽃받침길이', '꽃받침너비', '꽃잎길이', '꽃잎너비']
iris_features = pd.DataFrame(iris_features, columns=columns)
iris_features[:5]

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


레이블은 판다스의 시리즈로 변환한다.

In [47]:
iris_labels = pd.Series(iris_labels)
iris_labels

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Length: 150, dtype: object

150개의 데이터는 아래 세 개의 품종으로 구분되며, 각각 50개씩 아래 언급된 순서대로 구분되어 있다.

```
'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'
```

즉, 0번, 50번, 100번부터 각 품종의 데이터가 시작된다.
넘파이의 경우와는 달리 인덱스를 항상 함께 보여준다.

In [48]:
iris_labels[::50]

0          Iris-setosa
50     Iris-versicolor
100     Iris-virginica
dtype: object

In [49]:
iris_labels[:5]

0    Iris-setosa
1    Iris-setosa
2    Iris-setosa
3    Iris-setosa
4    Iris-setosa
dtype: object

In [50]:
iris_labels[50:55]

50    Iris-versicolor
51    Iris-versicolor
52    Iris-versicolor
53    Iris-versicolor
54    Iris-versicolor
dtype: object

In [51]:
iris_labels[100:105]

100    Iris-virginica
101    Iris-virginica
102    Iris-virginica
103    Iris-virginica
104    Iris-virginica
dtype: object

__연습 1.__ 꽃잎 길이(2번 열)가 1.5보다 크거나 꽃받침 길이(0번 열)가 5.0보다 작은 데이터만 추출하라.

부울 인덱싱은 넘파이의 경우와 기본적으로 동일하다.

In [52]:
mask = (iris_features.꽃잎길이>1.5) | (iris_features.꽃받침길이<5.0)
mask

0      False
1       True
2       True
3       True
4      False
       ...  
145     True
146     True
147     True
148     True
149     True
Length: 150, dtype: bool

조건을 만족하는 샘플의 수는 아래와 같다.

In [53]:
mask.sum()

129

조건을 만족하는 샘플들은 다음과 같다.
예를 들어 0번 4번 인덱스의 붓꽃은 조건에 맞지 않아 부울 인덱싱에서 걸러진다.
하지만 아래에서 볼 수 있듯이 다른 붓꽃의 인덱스가 조정되지는 않는다.
즉, 인덱스는 한 번 정해지면 절대 변하지 않는다.

In [54]:
iris_features[mask]

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


__연습 2.__ 꽃받침 길이(0번 열)와 꽃잎 길이(2번 열) 사이의 피어슨 상관계수를 계산하라.

힌트: 넘파이의 적절한 함수를 활용한다. 상관계수에 대한 설명은 [위키백과: 상관분석](https://ko.wikipedia.org/wiki/상관_분석)을 참고한다.
상관계수를 구하는 여러 방식이 있지만 판다스의 `corr()` 함수는 피어슨 상관계수를 기본으로 계산한다.
다른 방식의 상관계수를 구하려며 `method` 키워드 인자를 다르게 지정해야 한다. 
보다 자세한 사항은 [판다스: `corr()` 함수 문서](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)를 참조하라.

데이터프레임의 `corr()` 메서드는 모든 특성들 사이의 피어슨 상관계수로 이루어진 데이터프레임을 반환환다.

In [55]:
iris_corr = iris_features.corr()
iris_corr

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
꽃받침길이,1.000000,-0.109369,0.871754,0.817954
꽃받침너비,-0.109369,1.000000,-0.420516,-0.356544
꽃잎길이,0.871754,-0.420516,1.000000,0.962757
꽃잎너비,0.817954,-0.356544,0.962757,1.000000


따라서 '꽃받침길이'와 다른 특성들 사이의 상관계수를 역순으로 정렬하면 다음과 같다.

- 인덱싱: 데이터프레임의 인덱싱은 기본적으로 열(column) 단위로 이루어진다. 
    행(row) 단위 인덱싱은 `loc()` 또는 `iloc()` 메서드를 이용한다.
- `sort_values()`: 열 단위로 오름차순으로 정렬함. 역순으로 하고자 할 경우 `ascending=False` 키워드 인자 사용.

In [56]:
iris_corr['꽃받침길이'].sort_values(ascending=False)

꽃받침길이    1.000000
꽃잎길이     0.871754
꽃잎너비     0.817954
꽃받침너비   -0.109369
Name: 꽃받침길이, dtype: float64

따라서 '꽃받침길이'와 '꽃잎길이' 사이의 상관계수가 가장 높으며 
아래처럼 인덱싱을 두 번 사용하면 해당 값을 확인할 수 있다.

In [57]:
iris_corr['꽃받침길이']['꽃잎길이']

0.8717541573048719

__연습 3.__ 아래 식으로 계산된 값을 갖는 새로운 열(column)이 추가된 데이터프레임 `iris_features_added`를 생성하라.
열의 이름은 '길이속성1'으로 지정한다.

$$\frac{\text{원주율} \times \text{꽃잎길이} \times \text{꽃받침길이}^2}{3} $$

시리즈를 생성하면서 동시에 `name='길이특성1'` 이라는 키워드 인자를 사용하는 이유는
이어서 `iris_features` 데이터프레임과 합칠 때 새로 추가되는 열의 이름으로 
사용되도록 하기 위함이다.

In [58]:
# pass와 None을 각각 적절한 코드와 표현식으로 대체하라.

scaled = (3.14 * iris_features['꽃잎길이'] * iris_features['꽃받침길이']**2) / 3
length_property1 = pd.Series(scaled, name='길이특성1')

In [59]:
length_property1

0       38.113320
1       35.182653
2       30.057127
3       33.221200
4       36.633333
          ...    
145    244.321307
146    207.711000
147    229.952667
148    217.262880
149    185.815780
Name: 길이특성1, Length: 150, dtype: float64

In [60]:
iris_features_added = pd.concat([iris_features, length_property1], axis=1)

assert iris_features_added.shape == (150, 5)
iris_features_added

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비,길이특성1
0,5.1,3.5,1.4,0.2,38.113320
1,4.9,3.0,1.4,0.2,35.182653
2,4.7,3.2,1.3,0.2,30.057127
3,4.6,3.1,1.5,0.2,33.221200
4,5.0,3.6,1.4,0.2,36.633333
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,244.321307
146,6.3,2.5,5.0,1.9,207.711000
147,6.5,3.0,5.2,2.0,229.952667
148,6.2,3.4,5.4,2.3,217.262880


__연습 4.__ `Iris_versicolor` 품종에 해당하는 데이터만 `iris_features`로부터 추출하라. 

부울 인덱싱을 사용한다.

In [61]:
# None을 적절한 부울 표현식으로 대체하라.

mask = iris_labels == 'Iris-versicolor'
mask

0      False
1      False
2      False
3      False
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Length: 150, dtype: bool

정확히 50개의 샘플에 대해서만 `True`이다.

In [62]:
mask.sum()

50

보다 정확히는 50번부터 99번까지 붓꽃만 선택된다.

In [63]:
iris_versicolor = iris_features[mask]
iris_versicolor.head()

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
50,7.0,3.2,4.7,1.4
51,6.4,3.2,4.5,1.5
52,6.9,3.1,4.9,1.5
53,5.5,2.3,4.0,1.3
54,6.5,2.8,4.6,1.5


In [64]:
iris_versicolor.tail()

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
95,5.7,3.0,4.2,1.2
96,5.7,2.9,4.2,1.3
97,6.2,2.9,4.3,1.3
98,5.1,2.5,3.0,1.1
99,5.7,2.8,4.1,1.3


__연습 5.__ 꽃받침 길이(0번 열)의 평균값(mean), 중앙값(median), 표준편차(standard deviation)를 구하라.

__참고:__ 데이터프레임의 메서드는 기본적으로 열(columns)에 대한 속성을 다룬다.
즉, `axis=0`을 기본 축으로 사용한다.

In [65]:
iris_mean = iris_features.mean()
iris_mean

꽃받침길이    5.843333
꽃받침너비    3.054000
꽃잎길이     3.758667
꽃잎너비     1.198667
dtype: float64

In [66]:
iris_mean = iris_features.mean(axis=0)
iris_mean

꽃받침길이    5.843333
꽃받침너비    3.054000
꽃잎길이     3.758667
꽃잎너비     1.198667
dtype: float64

In [67]:
iris_median = iris_features.median()
iris_median

꽃받침길이    5.80
꽃받침너비    3.00
꽃잎길이     4.35
꽃잎너비     1.30
dtype: float64

In [68]:
iris_std = iris_features.std()
iris_std

꽃받침길이    0.828066
꽃받침너비    0.433594
꽃잎길이     1.764420
꽃잎너비     0.763161
dtype: float64

따라서 `for` 반복문을 이용하여 간단하게 세 개의 평균을 확인할 수 있다.

__참고:__ 특정 객체의 메소드로 이루어진 리스트에 포함된 메소드에 대한 반복문을 활용할 수 있다.
아래 코드는 통계와 관련해서 데이터프레임 객체가 제공하는 세 개의 메서드에 
대한 반복문을 적용하는 방식을 보여준다.

In [69]:
average_methods = [pd.DataFrame.mean, pd.DataFrame.median, pd.DataFrame.std]

for fun in average_methods:
    print(fun(iris_features)['꽃받침길이'], end=' ')

5.843333333333335 5.8 0.8280661279778629 

__연습 6.__ 세 개의 품종 별 꽃받침 너비(1번 열)의 평균값을 계산하여 아래 모양의  
데이터프레임과 시리즈(Series) `iris_mean_sepal_length`를 생성하라.

|                 | 평균 꽃받침 너비 |
| ---:            | ---:             |
| Iris-setosa     | 3.418            |
| Iris-versicolor | 2.770            |
| Iris-virginica  | 2.974            |


데이터프레임을 만들려면 `index`와 `columns` 키워드를 인자를 적절하게 지정해야 한다.

In [70]:
kinds = list(set(iris_labels))
kinds.sort()                      # 이름 순서를 맞추기 위해

iris_mean_sepal_width = []

for kind in kinds:
    mask = iris_labels == kind
    mean_0 = iris_features[mask].mean()['꽃받침너비']
    iris_mean_sepal_width.append(mean_0)
    
pd.DataFrame(iris_mean_sepal_width, index=kinds, columns=['평균 꽃받침 너비'])

,평균 꽃받침 너비
Iris-setosa,3.418
Iris-versicolor,2.770
Iris-virginica,2.974


시리즈를 만들려면 `index`와 `name` 키워드를 인자를 적절하게 지정해야 한다.

In [71]:
kinds = list(set(iris_labels))
kinds.sort()                      # 이름 순서를 맞추기 위해

iris_mean_sepal_width = []

for kind in kinds:
    mask = iris_labels == kind
    mean_0 = iris_features[mask].mean()['꽃받침너비']
    iris_mean_sepal_width.append(mean_0)
    
pd.Series(iris_mean_sepal_width, index=kinds, name='평균 꽃받침 너비')

Iris-setosa        3.418
Iris-versicolor    2.770
Iris-virginica     2.974
Name: 평균 꽃받침 너비, dtype: float64

__연습 7.__ 꽃잎 너비(3번 열)에 사용된 값을 모두 0과 1사이의 값으로 변환하라. 

힌트: 하나의 특성, 여기서는 꽃잎 너비,에 속하는 값을 모두 0과 1사이의 값으로 변환하는 작업을 정규화(normalization)이라 한다.
정규화에 대한 설명은 [정규화/표준화](https://rucrazia.tistory.com/90)을 참고하라.

In [72]:
iris_features[:5]

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


넘파이의 경우와 동일하게 작동한다.
하지만 데이터프레임의 메서드는 기본적으로 축을 0으로 지정해서 열 단위로 작동하기에
조금 더 간단하게 구현할 수 있다.

In [73]:
iris_features.min()

꽃받침길이    4.3
꽃받침너비    2.0
꽃잎길이     1.0
꽃잎너비     0.1
dtype: float64

In [74]:
iris_features.min(axis=0)

꽃받침길이    4.3
꽃받침너비    2.0
꽃잎길이     1.0
꽃잎너비     0.1
dtype: float64

In [75]:
iris_features_normalized = (iris_features - iris_features.min())/(iris_features.max() - iris_features.min())

iris_features_normalized

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667
...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667
146,0.555556,0.208333,0.677966,0.750000
147,0.611111,0.416667,0.711864,0.791667
148,0.527778,0.583333,0.745763,0.916667


이제 꽃잎 너비에 대한 정보만 인덱싱으로 추출하면 된다.

In [76]:
iris_features_normalized.꽃잎너비

0      0.041667
1      0.041667
2      0.041667
3      0.041667
4      0.041667
         ...   
145    0.916667
146    0.750000
147    0.791667
148    0.916667
149    0.708333
Name: 꽃잎너비, Length: 150, dtype: float64

__연습 8.__ `iris_features`에 사용된 모든 값을 특성 별로 표준화(standardization)하라. 

힌트: 표준화에 대한 설명은 [정규화/표준화](https://rucrazia.tistory.com/90)을 참고하라.

`mean()`, `std()` 메서드도 기본적으로 축을 0으로 지정해서 작동한다.

In [77]:
iris_features.mean()

꽃받침길이    5.843333
꽃받침너비    3.054000
꽃잎길이     3.758667
꽃잎너비     1.198667
dtype: float64

In [78]:
iris_features.std()

꽃받침길이    0.828066
꽃받침너비    0.433594
꽃잎길이     1.764420
꽃잎너비     0.763161
dtype: float64

In [79]:
# None을 적절한 부울 표현식으로 대체하라.

iris_features_standardized = (iris_features - iris_features.mean()) / iris_features.std()

iris_features_standardized[:5]

,꽃받침길이,꽃받침너비,꽃잎길이,꽃잎너비
0,-0.897674,1.028611,-1.336794,-1.308593
1,-1.139200,-0.124540,-1.336794,-1.308593
2,-1.380727,0.336720,-1.393470,-1.308593
3,-1.501490,0.106090,-1.280118,-1.308593
4,-1.018437,1.259242,-1.336794,-1.308593
